In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import KFold,train_test_split
# from sklearn import metrics
import seaborn as sns

plant_gen_1 = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv')
plant_weather_1 = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')
# convert UNIX time  
plant_gen_1['DATE_TIME_UNIX']     = pd.to_datetime(plant_gen_1['DATE_TIME']).astype(int) / 10**9                           
plant_weather_1['DATE_TIME_UNIX'] = pd.to_datetime(plant_weather_1['DATE_TIME']).astype(int) / 10**9
# plant_gen_1['DATE_TIME_UNIX']=pd.to_datetime(plant_gen_1['DATE_TIME_UNIX'], unit='s')


In [ ]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [ ]:
# merge
plant_1 = pd.merge(plant_gen_1, plant_weather_1, on='DATE_TIME_UNIX')


# remove any unused feature.
plant_1 = plant_1.drop(columns=['DATE_TIME_x','DATE_TIME_y', 
                              'PLANT_ID_x','PLANT_ID_y',
                              'SOURCE_KEY_x','SOURCE_KEY_y',
                              'DC_POWER',	'AC_POWER',
                              ])

# sort feature for easy to look


plant_1 = movecol(plant_1, 
             cols_to_move=['DAILY_YIELD','TOTAL_YIELD'], 
             ref_col='IRRADIATION',
             place='After')

plant_1

In [ ]:
plant_1=plant_1.groupby('DATE_TIME_UNIX').mean()
plant_1['DATE_TIME_UNIX'] = plant_1.index
plant_1 = movecol(plant_1, 
             cols_to_move=['DATE_TIME_UNIX'], 
             ref_col='AMBIENT_TEMPERATURE',
             place='Before')
plant_1

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(plant_1.DATE_TIME_UNIX,plant_1.DAILY_YIELD)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(plant_1.DATE_TIME_UNIX,plant_1.TOTAL_YIELD)

In [ ]:
# plant_1_roling=plant_1
plant_1_roling = pd.get_dummies(plant_1)
# roling
roling_num = 3

plant_1_roling['AMBIENT_TEMPERATURE_OLD_3']=(plant_1_roling['AMBIENT_TEMPERATURE'].rolling(roling_num+3).sum()-plant_1_roling['AMBIENT_TEMPERATURE'].rolling(roling_num).sum())/roling_num
plant_1_roling['MODULE_TEMPERATURE_OLD_3']=(plant_1_roling['MODULE_TEMPERATURE'].rolling(roling_num+3).sum()-plant_1_roling['MODULE_TEMPERATURE'].rolling(roling_num).sum())/roling_num
plant_1_roling['IRRADIATION_OLD_3']=(plant_1_roling['IRRADIATION'].rolling(roling_num+3).sum()-plant_1_roling['IRRADIATION'].rolling(roling_num).sum())/roling_num
plant_1_roling['DAILY_YIELD_OLD_3']=(plant_1_roling['TOTAL_YIELD'].rolling(roling_num+3).sum()-plant_1_roling['TOTAL_YIELD'].rolling(roling_num).sum())/roling_num
plant_1_roling['TOTAL_YIELD_OLD_3']=(plant_1_roling['TOTAL_YIELD'].rolling(roling_num+3).sum()-plant_1_roling['TOTAL_YIELD'].rolling(roling_num).sum())/roling_num

roling_num = 7
plant_1_roling['AMBIENT_TEMPERATURE_OLD_7']=(plant_1_roling['AMBIENT_TEMPERATURE'].rolling(roling_num+7).sum()-plant_1_roling['AMBIENT_TEMPERATURE'].rolling(roling_num).sum())/roling_num
plant_1_roling['MODULE_TEMPERATURE_OLD_7']=(plant_1_roling['MODULE_TEMPERATURE'].rolling(roling_num+7).sum()-plant_1_roling['MODULE_TEMPERATURE'].rolling(roling_num).sum())/roling_num
plant_1_roling['IRRADIATION_OLD_7']=(plant_1_roling['IRRADIATION'].rolling(roling_num+7).sum()-plant_1_roling['IRRADIATION'].rolling(roling_num).sum())/roling_num
plant_1_roling['DAILY_YIELD_OLD_7']=(plant_1_roling['TOTAL_YIELD'].rolling(roling_num+).sum()-plant_1_roling['TOTAL_YIELD'].rolling(roling_num).sum())/roling_num
plant_1_roling['TOTAL_YIELD_OLD_7']=(plant_1_roling['TOTAL_YIELD'].rolling(roling_num+7).sum()-plant_1_roling['TOTAL_YIELD'].rolling(roling_num).sum())/roling_num

# delete Nan
plant_1_roling.dropna(inplace=True)

# # invert index
# plant_1_roling = plant_1_roling.sort_index()

# plant_1_roling = movecol(plant_1_roling, 
#              cols_to_move=['AMBIENT_TEMPERATURE_OLD','MODULE_TEMPERATURE_OLD','IRRADIATION_OLD','DAILY_YIELD_OLD'], 
#              ref_col='AMBIENT_TEMPERATURE',
#              place='Before')

plant_1_roling

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
def K_fold(model, X,y, cv):

    kf = KFold(n_splits=cv, random_state=None, shuffle=True)
    kf.get_n_splits(X, y)

    rmse = list()
    mae = list()
    score= list()

    for train_index, test_index in kf.split(X):
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]

      m=model.fit(X_train.values, y_train.values.ravel())

      y_pred = np.round(model.predict(X_test))

      rmse.append(np.sqrt(mean_squared_error(y_test, y_pred)))
      mae.append(mean_absolute_error(y_test, y_pred))
      score.append(m.score(X, y))

    return np.mean(rmse),np.mean(mae),np.mean(score)

In [ ]:
# from sklearn.linear_model import LinearRegression,Ridge
from sklearn import linear_model
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')

def Regression (X,y,cv):
  LR  = linear_model.LinearRegression()
  RID = linear_model.Ridge(alpha=0.5)
  LL  = linear_model.LassoLars(alpha=.1)
  BR  = linear_model.BayesianRidge()
  DT  = tree.DecisionTreeRegressor()
  ET  = tree.ExtraTreeRegressor()


  print('Linear Regression    :',K_fold(LR,   X,y,cv))
  print('Ridge Regression     :',K_fold(RID,  X,y,cv)) 
  print('LassoLars            :',K_fold(LL,   X,y,cv)) 
  print('BayesianRidge        :',K_fold(BR,   X,y,cv)) 
  print('DecisionTreeRegressor:',K_fold(DT,   X,y,cv)) 
  print('ExtraTreeRegressor   :',K_fold(ET,   X,y,cv)) 
  print("(RMSE,MAE,Score)")

In [ ]:
print("Result")
print('__________________DATE_TIME_UNIX -> TOTAL_YIELD__________________')
X = pd.get_dummies(plant_1_roling[['DATE_TIME_UNIX']])
y = pd.get_dummies(plant_1_roling[['TOTAL_YIELD']])
Regression (X,y,10)
      
print("")
print('__________________OLD_3 -> TOTAL_YIELD__________________')
X = pd.get_dummies(plant_1_roling[[ 'AMBIENT_TEMPERATURE_OLD_3',
                                    'IRRADIATION_OLD_3',
                                    'MODULE_TEMPERATURE_OLD_3',
                                    'DAILY_YIELD_OLD_3',
                                    'TOTAL_YIELD_OLD_3']])
y = pd.get_dummies(plant_1_roling[['TOTAL_YIELD']])
Regression (X,y,10)


print("")
print('__________________OLD_7  -> TOTAL_YIELD__________________')
X = pd.get_dummies(plant_1_roling[[ 'AMBIENT_TEMPERATURE_OLD_7',
                                    'IRRADIATION_OLD_7',
                                    'MODULE_TEMPERATURE_OLD_7',
                                    'DAILY_YIELD_OLD_7',
                                    'TOTAL_YIELD_OLD_7']])
y = pd.get_dummies(plant_1_roling[['TOTAL_YIELD']])
Regression (X,y,10)


print("")
print('__________________DATE_TIME_UNIX + OLD_3 -> TOTAL_YIELD__________________')
X = pd.get_dummies(plant_1_roling[[ 'DATE_TIME_UNIX',
                                    'AMBIENT_TEMPERATURE_OLD_3',
                                    'IRRADIATION_OLD_3',
                                    'MODULE_TEMPERATURE_OLD_3',
                                    'DAILY_YIELD_OLD_3',
                                    'TOTAL_YIELD_OLD_3']])
y = pd.get_dummies(plant_1_roling[['TOTAL_YIELD']])
Regression (X,y,10)


print("")
print('__________________DATE_TIME_UNIX + OLD_7  -> TOTAL_YIELD__________________')
X = pd.get_dummies(plant_1_roling[['DATE_TIME_UNIX',
                            'AMBIENT_TEMPERATURE_OLD_7',
                            'IRRADIATION_OLD_7',
                            'MODULE_TEMPERATURE_OLD_7',
                            'DAILY_YIELD_OLD_7',
                            'TOTAL_YIELD_OLD_7']])
y = pd.get_dummies(plant_1_roling[['TOTAL_YIELD']])
Regression (X,y,10)